# Understanding the Satellites Domain: A Bilevel Learning Case Study

This notebook provides a comprehensive introduction to the Satellites domain used in IVNTR (Bilevel Learning for Bilevel Planning). We'll explore how this domain demonstrates the key challenges and solutions in learning neural predicates for symbolic planning.

## Overview

The Satellites domain is a 2D continuous environment where satellites equipped with different instruments must take readings of objects. This domain perfectly illustrates:

1. **Continuous-to-Symbolic Abstraction**: Moving from continuous positions/orientations to discrete relational predicates
2. **Multi-step Coordination**: Requiring complex temporal planning across multiple agents 
3. **Partial Observability**: Learning predicates that capture important but hidden relationships

Let's dive in!

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt

# Add the project root to path
sys.path.append('..')

# Import IVNTR components
from predicators.envs.satellites import SatellitesEnv
from predicators import utils
from predicators.settings import CFG

# Configure for tutorial
utils.reset_config({
    "env": "satellites",
})

CFG.seed = 42
CFG.num_train_tasks = 3
CFG.num_test_tasks = 2
CFG.satellites_num_sat_train = [2, 3]
CFG.satellites_num_obj_train = [2, 3]
CFG.satellites_num_sat_test = [2, 3]
CFG.satellites_num_obj_test = [2, 3]

print("✅ Successfully imported IVNTR components!")

/Users/libowen/opt/anaconda3/envs/ivntr/lib/python3.8/site-packages/bosdyn/client/sdk.py:17: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/Users/libowen/opt/anaconda3/envs/ivntr/lib/python3.8/site-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/libowen/opt/anaconda3/envs/ivntr/lib/python3.8/site-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.ai')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest

✅ Successfully imported IVNTR components!


## 1. Domain Structure and Entities

The Satellites domain consists of two main entity types:

### Satellites
Each satellite has the following features:
- **Position**: (x, y) coordinates in [0,1] × [0,1] space
- **Orientation**: θ angle determining viewing direction
- **Instrument**: Camera (0.0-0.33), Infrared (0.33-0.66), or Geiger (0.66-1.0)
- **Capabilities**: Can shoot Chemical X and/or Y
- **Calibration**: Must be calibrated against specific objects before taking readings
- **Field of View**: Triangular viewing cone

### Objects
Each object has:
- **Position**: (x, y) coordinates
- **ID**: Unique identifier
- **Chemical State**: Whether it has been shot with Chemical X or Y

In [2]:
# Create the satellites environment
env = SatellitesEnv(use_gui=False)

print("🚀 Satellites Environment Created")
print(f"Environment name: {env.get_name()}")
print(f"Action space: {env.action_space}")
print("\n📊 Types in the domain:")
for type_obj in env.types:
    print(f"  - {type_obj.name}: {type_obj.feature_names}")

print("\n🔍 Predicates in the domain:")
for pred in sorted(env.predicates, key=lambda p: p.name):
    types_str = ", ".join([t.name for t in pred.types])
    print(f"  - {pred.name}({types_str})")

print("\n🎯 Goal predicates (what we're trying to achieve):")
for pred in sorted(env.goal_predicates, key=lambda p: p.name):
    types_str = ", ".join([t.name for t in pred.types])
    print(f"  - {pred.name}({types_str})")

🚀 Satellites Environment Created
Environment name: satellites
Action space: Box(0.0, 1.0, (10,), float32)

📊 Types in the domain:
  - object: ['id', 'x', 'y', 'has_chem_x', 'has_chem_y']
  - satellite: ['x', 'y', 'theta', 'instrument', 'calibration_obj_id', 'is_calibrated', 'read_obj_id', 'shoots_chem_x', 'shoots_chem_y']

🔍 Predicates in the domain:
  - CalibrationTarget(satellite, object)
  - CameraReadingTaken(satellite, object)
  - GeigerReadingTaken(satellite, object)
  - HasCamera(satellite)
  - HasChemX(object)
  - HasChemY(object)
  - HasGeiger(satellite)
  - HasInfrared(satellite)
  - InfraredReadingTaken(satellite, object)
  - IsCalibrated(satellite)
  - Sees(satellite, object)
  - ShootsChemX(satellite)
  - ShootsChemY(satellite)
  - ViewClear(satellite)

🎯 Goal predicates (what we're trying to achieve):
  - CameraReadingTaken(satellite, object)
  - GeigerReadingTaken(satellite, object)
  - InfraredReadingTaken(satellite, object)


## 2. Task Generation and Visualization

Let's generate a sample task to understand the domain better. Each task consists of:
- An initial state with satellites and objects positioned randomly
- A goal requiring specific satellites to take readings of specific objects

In [3]:
# Generate a sample training task
train_tasks = env.get_train_tasks()
sample_task = train_tasks[0]

print("📋 Sample Task Generated")
print(f"Entities in initial state: {sample_task.init.data.keys()}")

# Analyze the initial state
satellites = list(sample_task.init.get_objects(env._sat_type))
objects = list(sample_task.init.get_objects(env._obj_type))

print(f"\n🛸 Satellites ({satellites}):")
for i, sat in enumerate(satellites):
    x = sample_task.init.get(sat, "x")
    y = sample_task.init.get(sat, "y")
    instrument = sample_task.init.get(sat, "instrument")
    calibration_obj = int(sample_task.init.get(sat, "calibration_obj_id"))
    shoots_x = sample_task.init.get(sat, "shoots_chem_x") > 0.5
    shoots_y = sample_task.init.get(sat, "shoots_chem_y") > 0.5
    
    # Determine instrument type
    if instrument <= 0.33:
        inst_type = "Camera"
    elif instrument <= 0.66:
        inst_type = "Infrared"
    else:
        inst_type = "Geiger"
    
    print(f"  {sat.name}: pos=({x:.2f}, {y:.2f}), {inst_type}, cal_obj={calibration_obj}, ChemX={shoots_x}, ChemY={shoots_y}")

print(f"\n🎯 Objects ({objects}):")
for obj in objects:
    x = sample_task.init.get(obj, "x")
    y = sample_task.init.get(obj, "y")
    obj_id = int(sample_task.init.get(obj, "id"))
    has_chem_x = sample_task.init.get(obj, "has_chem_x") > 0.5
    has_chem_y = sample_task.init.get(obj, "has_chem_y") > 0.5
    print(f"  {obj.name} (ID {obj_id}): pos=({x:.2f}, {y:.2f}), ChemX={has_chem_x}, ChemY={has_chem_y}")

print(f"\n🏆 Goals ({sample_task.goal}):")
for goal_atom in sample_task.goal:
    print(f"  {goal_atom}")

📋 Sample Task Generated
Entities in initial state: dict_keys([sat0:satellite, sat1:satellite, obj0:object, obj1:object, obj2:object])

🛸 Satellites ([sat0:satellite, sat1:satellite]):
  sat0: pos=(0.44, 0.86), Camera, cal_obj=1, ChemX=True, ChemY=True
  sat1: pos=(0.79, 0.13), Infrared, cal_obj=2, ChemX=False, ChemY=True

🎯 Objects ([obj0:object, obj1:object, obj2:object]):
  obj0 (ID 0): pos=(0.44, 0.23), ChemX=False, ChemY=False
  obj1 (ID 1): pos=(0.83, 0.63), ChemX=False, ChemY=False
  obj2 (ID 2): pos=(0.15, 0.68), ChemX=False, ChemY=False

🏆 Goals ({CameraReadingTaken(sat0:satellite, obj2:object), InfraredReadingTaken(sat1:satellite, obj2:object)}):
  CameraReadingTaken(sat0:satellite, obj2:object)
  InfraredReadingTaken(sat1:satellite, obj2:object)


In [ ]:
# Visualize the sample task
fig = env.render_state_plt(
    sample_task.init, 
    sample_task,
    caption="Sample Satellites Task - Initial State"
)
plt.show()


🖼️ Visualization Legend:
  • Red circles: Uncalibrated satellites (haven't taken readings)
  • Blue circles: Calibrated satellites (ready to take readings)
  • Green circles: Satellites that have taken readings
  • Black circles: Objects
  • Purple triangles: Satellite field-of-view cones
  • Hatched objects: Have chemical X and/or Y


/Users/libowen/opt/anaconda3/envs/ivntr/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 3. Understanding Domain Predicates

The domain contains several types of predicates that capture different aspects of the satellite mission:

### Static Predicates (Never Change)
- `CalibrationTarget(satellite, object)`: Which object each satellite should calibrate against
- `HasCamera/HasInfrared/HasGeiger(satellite)`: What instrument each satellite carries
- `ShootsChemX/ShootsChemY(satellite)`: What chemicals each satellite can shoot

### Dynamic Predicates (Change During Execution)
- `Sees(satellite, object)`: Whether satellite can see object (geometric relationship)
- `IsCalibrated(satellite)`: Whether satellite is calibrated
- `HasChemX/HasChemY(object)`: Whether object has been shot with chemicals
- `ViewClear(satellite)`: Whether satellite's view is unobstructed

### Goal Predicates (What We Want to Achieve)
- `CameraReadingTaken(satellite, object)`
- `InfraredReadingTaken(satellite, object)`  
- `GeigerReadingTaken(satellite, object)`

In [10]:
# Let's analyze which predicates hold in our sample task
state = sample_task.init

print("🔍 Predicate Analysis for Sample Task\n")

# Check static predicates
print("📌 Static Predicates:")
for sat in satellites:
    for obj in objects:
        if env._CalibrationTarget_holds(state, [sat, obj]):
            print(f"  ✓ CalibrationTarget({sat.name}, {obj.name})")
    
    if env._HasCamera_holds(state, [sat]):
        print(f"  ✓ HasCamera({sat.name})")
    elif env._HasInfrared_holds(state, [sat]):
        print(f"  ✓ HasInfrared({sat.name})")
    elif env._HasGeiger_holds(state, [sat]):
        print(f"  ✓ HasGeiger({sat.name})")
    
    if env._ShootsChemX_holds(state, [sat]):
        print(f"  ✓ ShootsChemX({sat.name})")
    if env._ShootsChemY_holds(state, [sat]):
        print(f"  ✓ ShootsChemY({sat.name})")

# Check dynamic predicates
print("\n⚡ Dynamic Predicates:")
for sat in satellites:
    if env._IsCalibrated_holds(state, [sat]):
        print(f"  ✓ IsCalibrated({sat.name})")
    else:
        print(f"  ✗ IsCalibrated({sat.name})")
    
    if env._ViewClear_holds(state, [sat]):
        print(f"  ✓ ViewClear({sat.name})")
    else:
        print(f"  ✗ ViewClear({sat.name})")
    
    for obj in objects:
        if env._Sees_holds(state, [sat, obj]):
            print(f"  ✓ Sees({sat.name}, {obj.name})")

for obj in objects:
    if env._HasChemX_holds(state, [obj]):
        print(f"  ✓ HasChemX({obj.name})")
    if env._HasChemY_holds(state, [obj]):
        print(f"  ✓ HasChemY({obj.name})")

# Check goal predicates (should all be false initially)
print("\n🏆 Goal Predicates (should be false initially):")
goal_checks = 0
for sat in satellites:
    for obj in objects:
        if env._CameraReadingTaken_holds(state, [sat, obj]):
            print(f"  ✓ CameraReadingTaken({sat.name}, {obj.name})")
            goal_checks += 1
        if env._InfraredReadingTaken_holds(state, [sat, obj]):
            print(f"  ✓ InfraredReadingTaken({sat.name}, {obj.name})")
            goal_checks += 1
        if env._GeigerReadingTaken_holds(state, [sat, obj]):
            print(f"  ✓ GeigerReadingTaken({sat.name}, {obj.name})")
            goal_checks += 1

if goal_checks == 0:
    print("  (None - as expected for initial state)")

🔍 Predicate Analysis for Sample Task

📌 Static Predicates:
  ✓ CalibrationTarget(sat0, obj1)
  ✓ HasCamera(sat0)
  ✓ ShootsChemX(sat0)
  ✓ ShootsChemY(sat0)
  ✓ CalibrationTarget(sat1, obj2)
  ✓ HasInfrared(sat1)
  ✓ ShootsChemY(sat1)

⚡ Dynamic Predicates:
  ✗ IsCalibrated(sat0)
  ✓ ViewClear(sat0)
  ✗ IsCalibrated(sat1)
  ✓ ViewClear(sat1)

🏆 Goal Predicates (should be false initially):
  (None - as expected for initial state)


## 4. Action Space and Dynamics

The satellites domain has a 10-dimensional continuous action space:

`[cur_sat_x, cur_sat_y, obj_x, obj_y, target_sat_x, target_sat_y, calibrate, shoot_chem_x, shoot_chem_y, use_instrument]`

The action interpretation depends on which flag is set:
- **Movement**: If no flags are set, satellite moves to target position
- **Calibration**: If `calibrate > 0.5`, satellite calibrates against object
- **Chemical Shooting**: If `shoot_chem_x/y > 0.5`, satellite shoots chemical at object
- **Instrument Use**: If `use_instrument > 0.5`, satellite takes reading of object

In [11]:
# Let's demonstrate different types of actions
from predicators.structs import Action

print("🎮 Action Space Demonstration\n")

# Get current state info
sat = satellites[0]
obj = objects[0]
sat_x = state.get(sat, "x")
sat_y = state.get(sat, "y")
obj_x = state.get(obj, "x")
obj_y = state.get(obj, "y")

print(f"Initial: {sat.name} at ({sat_x:.3f}, {sat_y:.3f}), {obj.name} at ({obj_x:.3f}, {obj_y:.3f})")

# 1. Movement action
target_x, target_y = 0.3, 0.7  # Move to new position
move_action = Action(np.array([
    sat_x, sat_y, obj_x, obj_y, target_x, target_y,
    0.0, 0.0, 0.0, 0.0  # no special actions
], dtype=np.float32))

new_state = env.simulate(state, move_action)
new_sat_x = new_state.get(sat, "x")
new_sat_y = new_state.get(sat, "y")
print(f"\n🚀 Movement Action:")
print(f"  Before: {sat.name} at ({sat_x:.3f}, {sat_y:.3f})")
print(f"  After:  {sat.name} at ({new_sat_x:.3f}, {new_sat_y:.3f})")

# 2. Check if satellite can see object after movement
can_see = env._Sees_holds(new_state, [sat, obj])
print(f"  Can see {obj.name}: {can_see}")

# 3. Calibration action (if possible)
if env._CalibrationTarget_holds(new_state, [sat, obj]) and can_see:
    calibrate_action = Action(np.array([
        new_sat_x, new_sat_y, obj_x, obj_y, new_sat_x, new_sat_y,
        1.0, 0.0, 0.0, 0.0  # calibrate flag set
    ], dtype=np.float32))
    
    calibrated_state = env.simulate(new_state, calibrate_action)
    is_calibrated = env._IsCalibrated_holds(calibrated_state, [sat])
    
    print(f"\n🔧 Calibration Action:")
    print(f"  Calibrated: {is_calibrated}")
    
    state_for_next = calibrated_state
else:
    print(f"\n🔧 Calibration Action: Not possible (wrong target or can't see)")
    state_for_next = new_state

# 4. Chemical shooting (if satellite has the capability)
if env._ShootsChemX_holds(state_for_next, [sat]) and env._Sees_holds(state_for_next, [sat, obj]):
    shoot_action = Action(np.array([
        new_sat_x, new_sat_y, obj_x, obj_y, new_sat_x, new_sat_y,
        0.0, 1.0, 0.0, 0.0  # shoot ChemX flag set
    ], dtype=np.float32))
    
    shot_state = env.simulate(state_for_next, shoot_action)
    has_chem_x = env._HasChemX_holds(shot_state, [obj])
    
    print(f"\n💥 Chemical X Shooting:")
    print(f"  {obj.name} has ChemX: {has_chem_x}")
else:
    print(f"\n💥 Chemical X Shooting: Not possible (satellite can't shoot ChemX or can't see object)")

🎮 Action Space Demonstration

Initial: sat0 at (0.439, 0.859), obj0 at (0.443, 0.227)

🚀 Movement Action:
  Before: sat0 at (0.439, 0.859)
  After:  sat0 at (0.300, 0.700)
  Can see obj0: False

🔧 Calibration Action: Not possible (wrong target or can't see)

💥 Chemical X Shooting: Not possible (satellite can't shoot ChemX or can't see object)


## Summary

In this notebook, we've explored the Satellites domain in detail:

### 🛸 **Domain Structure**
- **Satellites**: Position, orientation, instruments (Camera/Infrared/Geiger), chemical capabilities
- **Objects**: Position, ID, chemical states
- **Goals**: Take specific readings with specific satellites

### 🔍 **Predicate System**
- **Static predicates**: Capabilities and target relationships that never change
- **Dynamic predicates**: Spatial relationships, calibration, and chemical states that evolve
- **Goal predicates**: The readings we want to achieve

### 🎮 **Action Space**
- 10-dimensional continuous actions covering movement, calibration, chemical shooting, and instrument use
- Complex state transitions requiring precise sequencing

### 🤝 **Coordination Challenges**
- Multi-step dependencies: positioning → seeing → calibration → chemical preparation → reading
- Multi-agent coordination: Different satellites have different capabilities
- Temporal planning: Must sequence actions correctly across time

This domain provides the perfect testbed for bilevel learning because it requires both:
1. **Learning abstract relationships** from continuous features (the neural challenge)
2. **Symbolic planning** with those relationships (the symbolic challenge)

---

**Next: `02_planner.ipynb` - Ground Truth Planning Model**